In [1]:
import numpy as np
from sklearn.metrics import roc_curve
from sklearn.neighbors import KNeighborsClassifier
import random
import matplotlib.patheffects as PathEffects

from keras.layers import Input, Conv2D, Lambda, Dense, Flatten,MaxPooling2D, concatenate
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from tensorflow.keras.optimizers import SGD
from keras.losses import binary_crossentropy
import os
import pickle
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam

from itertools import permutations

import seaborn as sns

from keras.datasets import mnist
from sklearn.manifold import TSNE

from sklearn.svm import SVC


In [2]:
import torch
from sklearn.model_selection import train_test_split
import pandas as pd

In [3]:
with open('Data/sub_dataset_list.pkl', 'rb') as f:
    smellKey_list = pickle.load(f)
smellKey_list

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
95    14.0
96    14.0
97    14.0
98    14.0
99    14.0
Name: smellKey_encoded, Length: 100, dtype: float64

In [4]:
save_path = 'Data/fn_smell_embeddings_with_triplet.pt'
loaded_ast_embeddings = torch.load(save_path)
loaded_ast_embeddings = loaded_ast_embeddings.numpy()
loaded_ast_embeddings

array([[ 0.05305316,  0.721399  ,  0.5732451 , ..., -0.82329357,
        -0.9310498 ,  0.29698396],
       [-0.22184192,  0.23973262,  0.3208173 , ..., -0.5386479 ,
        -0.3320793 ,  0.83964026],
       [ 0.03883142,  0.33044964,  1.1247997 , ...,  0.07288593,
        -0.9057045 ,  0.12496915],
       ...,
       [-0.16270329, -0.26236066,  0.76572466, ...,  0.14703672,
        -0.08933374,  0.04029626],
       [ 0.16533531,  0.9115994 ,  0.6147458 , ..., -0.11640675,
        -0.6950825 ,  0.14291163],
       [ 0.39961746,  0.5364607 ,  0.41132835, ..., -0.2791034 ,
        -0.22381859,  0.05056619]], dtype=float32)

In [5]:
data = pd.DataFrame({
    'fn_smell_embeddings.pt': loaded_ast_embeddings.tolist(),
    'smellKey_encoded': smellKey_list
})

X = np.array(data['fn_smell_embeddings.pt'].tolist())
y = np.array(data['smellKey_encoded'])

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [6]:
x_test.shape

(20, 768)

In [7]:
# Define our own plot function
def scatter(x, labels, subtitle=None):
    # We choose a color palette with seaborn.
    palette = np.array(sns.color_palette("hls", 10))

    # We create a scatter plot.
    f = plt.figure(figsize=(8, 8))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(x[:,0], x[:,1], lw=0, s=40,
                    c=palette[labels.astype(int)])
    plt.xlim(-25, 25)
    plt.ylim(-25, 25)
    ax.axis('off')
    ax.axis('tight')

    # We add the labels for each digit.
    txts = []
    for i in range(10):
        # Position of each label.
        xtext, ytext = np.median(x[labels == i, :], axis=0)
        txt = ax.text(xtext, ytext, str(i), fontsize=24)
        txt.set_path_effects([
            PathEffects.Stroke(linewidth=5, foreground="w"),
            PathEffects.Normal()])
        txts.append(txt)
        
    if subtitle != None:
        plt.suptitle(subtitle)
        
    plt.savefig(subtitle)

In [8]:
x_train_flat = x_train
x_test_flat = x_test
print(x_train.shape)
print(x_train_flat.shape)

(80, 768)
(80, 768)


In [ ]:
tsne = TSNE()
train_tsne_embeds = tsne.fit_transform(x_train[:512])
scatter(train_tsne_embeds, y_train[:512], "Samples from Training Data")

eval_tsne_embeds = tsne.fit_transform(x_test[:512])
scatter(eval_tsne_embeds, y_test[:512], "Samples from Validation Data")

In [9]:
Classifier_input = Input((768,))
Classifier_output = Dense(10, activation='softmax')(Classifier_input)
Classifier_model = Model(Classifier_input, Classifier_output)

In [10]:
from sklearn.preprocessing import LabelBinarizer

In [11]:
le = LabelBinarizer()

In [12]:
y_train_onehot = le.fit_transform(y_train)
y_test_onehot = le.transform(y_test)

In [13]:
Classifier_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [14]:
Classifier_model.fit(x_train_flat,y_train_onehot, validation_data=(x_test_flat,y_test_onehot),epochs=50)

Epoch 1/50
3/3 [==============================] - 1s 77ms/step - loss: 2.7072 - accuracy: 0.1125 - val_loss: 2.2987 - val_accuracy: 0.1000
Epoch 2/50
3/3 [==============================] - 0s 12ms/step - loss: 2.3014 - accuracy: 0.1750 - val_loss: 2.2825 - val_accuracy: 0.1500
Epoch 3/50
3/3 [==============================] - 0s 14ms/step - loss: 2.1569 - accuracy: 0.2875 - val_loss: 2.3193 - val_accuracy: 0.2500
Epoch 4/50
3/3 [==============================] - 0s 17ms/step - loss: 2.0579 - accuracy: 0.3375 - val_loss: 2.3429 - val_accuracy: 0.1000
Epoch 5/50
3/3 [==============================] - 0s 15ms/step - loss: 1.9717 - accuracy: 0.3875 - val_loss: 2.3260 - val_accuracy: 0.1000
Epoch 6/50
3/3 [==============================] - 0s 11ms/step - loss: 1.8763 - accuracy: 0.4375 - val_loss: 2.2385 - val_accuracy: 0.2500
Epoch 7/50
3/3 [==============================] - 0s 12ms/step - loss: 1.7757 - accuracy: 0.4500 - val_loss: 2.1284 - val_accuracy: 0.2500
Epoch 8/50
3/3 [===========

In [15]:
def generate_triplet(x,y,testsize=0.3,ap_pairs=10,an_pairs=10):
    data_xy = tuple([x,y])

    trainsize = 1-testsize

    triplet_train_pairs = []
    triplet_test_pairs = []
    for data_class in sorted(set(data_xy[1])):

        same_class_idx = np.where((data_xy[1] == data_class))[0]
        diff_class_idx = np.where(data_xy[1] != data_class)[0]
        A_P_pairs = random.sample(list(permutations(same_class_idx,2)),k=ap_pairs) #Generating Anchor-Positive pairs
        Neg_idx = random.sample(list(diff_class_idx),k=an_pairs)
        

        #train
        A_P_len = len(A_P_pairs)
        Neg_len = len(Neg_idx)
        for ap in A_P_pairs[:int(A_P_len*trainsize)]:
            Anchor = data_xy[0][ap[0]]
            Positive = data_xy[0][ap[1]]
            for n in Neg_idx:
                Negative = data_xy[0][n]
                triplet_train_pairs.append([Anchor,Positive,Negative])               
        #test
        for ap in A_P_pairs[int(A_P_len*trainsize):]:
            Anchor = data_xy[0][ap[0]]
            Positive = data_xy[0][ap[1]]
            for n in Neg_idx:
                Negative = data_xy[0][n]
                triplet_test_pairs.append([Anchor,Positive,Negative])    
                
    return np.array(triplet_train_pairs), np.array(triplet_test_pairs)

In [16]:
X_train, X_test = generate_triplet(x_train_flat,y_train, ap_pairs=30, an_pairs=30,testsize=0.2)
X_train.shape

(7200, 3, 768)

## Triplet NN

In [17]:
def triplet_loss(y_true, y_pred, alpha = 0.4):
    """
    Implementation of the triplet loss function
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor data
            positive -- the encodings for the positive data (similar to anchor)
            negative -- the encodings for the negative data (different from anchor)
    Returns:
    loss -- real number, value of the loss
    """
    print('y_pred.shape = ',y_pred)
    
    total_lenght = y_pred.shape.as_list()[-1]
#     print('total_lenght=',  total_lenght)
#     total_lenght =12
    
    anchor = y_pred[:,0:int(total_lenght*1/3)]
    positive = y_pred[:,int(total_lenght*1/3):int(total_lenght*2/3)]
    negative = y_pred[:,int(total_lenght*2/3):int(total_lenght*3/3)]

    # distance between the anchor and the positive
    pos_dist = K.sum(K.square(anchor-positive),axis=1)

    # distance between the anchor and the negative
    neg_dist = K.sum(K.square(anchor-negative),axis=1)

    # compute loss
    basic_loss = pos_dist-neg_dist+alpha
    loss = K.maximum(basic_loss,0.0)
 
    return loss

In [18]:
def create_base_network():
    """
    Base network to be shared.
    """
    model = Sequential()
    model.add(Flatten(input_shape=(768,1)))
    # model.add(Conv2D(128,(7,7),padding='same',input_shape=(in_dims[0],in_dims[1],in_dims[2],),activation='relu',name='conv1'))
    # model.add(MaxPooling2D((2,2),(2,2),padding='same',name='pool1'))
    # model.add(Conv2D(256,(5,5),padding='same',activation='relu',name='conv2'))
    # model.add(MaxPooling2D((2,2),(2,2),padding='same',name='pool2'))
    # model.add(Flatten(name='flatten'))
    model.add(Dense(768,name='embeddings'))
    # model.add(Dense(600))
    
    return model

In [19]:
adam_optim = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999)

In [20]:
anchor_input = Input((768,1), name='anchor_input')
positive_input = Input((768,1 ), name='positive_input')
negative_input = Input((768,1 ), name='negative_input')

# Shared embedding layer for positive and negative items
Shared_DNN = create_base_network()


encoded_anchor = Shared_DNN(anchor_input)
encoded_positive = Shared_DNN(positive_input)
encoded_negative = Shared_DNN(negative_input)


merged_vector = concatenate([encoded_anchor, encoded_positive, encoded_negative], axis=-1, name='merged_layer')

model = Model(inputs=[anchor_input,positive_input, negative_input], outputs=merged_vector)
model.compile(loss=triplet_loss, optimizer=adam_optim)

In [21]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 anchor_input (InputLayer)      [(None, 768, 1)]     0           []                               
                                                                                                  
 positive_input (InputLayer)    [(None, 768, 1)]     0           []                               
                                                                                                  
 negative_input (InputLayer)    [(None, 768, 1)]     0           []                               
                                                                                                  
 sequential (Sequential)        (None, 768)          590592      ['anchor_input[0][0]',           
                                                                  'positive_input[0][0]',   

In [22]:
X_train.shape

(7200, 3, 768)

In [44]:
X_train[1,0,:] == X_train[0,0,:]



array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [43]:
Anchor = X_train[:,0,:].reshape(-1,768,1)
Positive = X_train[:,1,:].reshape(-1,768,1)
Negative = X_train[:,2,:].reshape(-1,768,1)
Anchor_test = X_test[:,0,:].reshape(-1,768,1)
Positive_test = X_test[:,1,:].reshape(-1,768,1)
Negative_test = X_test[:,2,:].reshape(-1,768,1)

Y_dummy = np.empty((Anchor.shape[0],300))
Y_dummy2 = np.empty((Anchor_test.shape[0],1))

model.fit([Anchor,Positive,Negative],y=Y_dummy,validation_data=([Anchor_test,Positive_test,Negative_test],Y_dummy2), batch_size=512, epochs=500)

Epoch 1/500
y_pred.shape =  Tensor("model_1/merged_layer/concat:0", shape=(None, 2304), dtype=float32)
y_pred.shape =  Tensor("model_1/merged_layer/concat:0", shape=(None, 2304), dtype=float32)
13/15 [=========================>....] - ETA: 0s - loss: 2.1601y_pred.shape =  Tensor("model_1/merged_layer/concat:0", shape=(None, 2304), dtype=float32)


c:\Users\omerkerem.adali\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\python\framework\constant_op.py:103: RuntimeWarning: overflow encountered in cast
  return ops.EagerTensor(value, ctx.device_name, dtype)


15/15 [==============================] - 1s 51ms/step - loss: 2.0100 - val_loss: 1.2054
Epoch 2/500
15/15 [==============================] - 1s 42ms/step - loss: 0.1395 - val_loss: 0.9854
Epoch 3/500
15/15 [==============================] - 1s 37ms/step - loss: 0.0817 - val_loss: 0.7565
Epoch 4/500
15/15 [==============================] - 1s 38ms/step - loss: 0.0453 - val_loss: 0.5119
Epoch 5/500
15/15 [==============================] - 1s 38ms/step - loss: 0.0439 - val_loss: 0.4505
Epoch 6/500
15/15 [==============================] - 1s 36ms/step - loss: 0.0395 - val_loss: 0.4647
Epoch 7/500
15/15 [==============================] - 1s 37ms/step - loss: 0.0341 - val_loss: 0.4396
Epoch 8/500
15/15 [==============================] - 1s 38ms/step - loss: 0.0252 - val_loss: 0.5060
Epoch 9/500
15/15 [==============================] - 1s 37ms/step - loss: 0.0200 - val_loss: 0.4621
Epoch 10/500
15/15 [==============================] - 1s 38ms/step - loss: 0.0195 - val_loss: 0.4540
Epoch 11/50

In [45]:
predictedModel = model.predict([Anchor, Positive, Negative])

225/225 [==============================] - 1s 3ms/step


In [46]:
predictedModel.shape


(7200, 2304)

In [48]:
outputAnchor = predictedModel[:, :768]
outputPositive = predictedModel[:, 768:1536]
outputNegative = predictedModel[:, 1536:2304]

In [49]:
outputAnchor.shape

(7200, 768)